In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 185, 185

top_model_weights_path = 'bottleneck_fc_2_model.h5'
train_data_dir = 'Posters 660/Train'
validation_data_dir = 'Posters 660/Validation'
nb_train_samples = 396
nb_validation_samples = 264
epochs = 50
batch_size = 36 # difference in this version: train the VGG using a larger batch size


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_2_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_2_validation.npy', 'w'),
            bottleneck_features_validation)

save_bottlebeck_features()

Using TensorFlow backend.


Found 396 images belonging to 3 classes.
Found 264 images belonging to 3 classes.


In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras import optimizers

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'Posters 660/Train'
validation_data_dir = 'Posters 660/Validation'
nb_train_samples = 396
nb_validation_samples = 264
epochs = 125
batch_size = 36 

sgd = SGD(lr=0.0001, momentum=0.9) # lower learning rate

def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    
#add more layers, eliminate dropout
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu')) 
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=sgd,
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

train_top_model()

Train on 396 samples, validate on 264 samples
Epoch 1/125
396/396 [==============================] - 1s - loss: 0.7035 - acc: 0.5076 - val_loss: 0.6913 - val_acc: 0.5265
Epoch 2/125
396/396 [==============================] - 0s - loss: 0.6955 - acc: 0.5025 - val_loss: 0.6908 - val_acc: 0.5417
Epoch 3/125
396/396 [==============================] - 0s - loss: 0.6967 - acc: 0.4949 - val_loss: 0.6903 - val_acc: 0.5644
Epoch 4/125
396/396 [==============================] - 0s - loss: 0.6873 - acc: 0.5556 - val_loss: 0.6902 - val_acc: 0.5568
Epoch 5/125
396/396 [==============================] - 0s - loss: 0.6934 - acc: 0.4975 - val_loss: 0.6898 - val_acc: 0.5720
Epoch 6/125
396/396 [==============================] - 0s - loss: 0.6910 - acc: 0.5202 - val_loss: 0.6892 - val_acc: 0.5644
Epoch 7/125
396/396 [==============================] - 0s - loss: 0.6926 - acc: 0.5126 - val_loss: 0.6888 - val_acc: 0.5682
Epoch 8/125
396/396 [==============================] - 0s - loss: 0.6909 - acc: 0.5530

In [20]:
train_data = np.load(open('bottleneck_features_train.npy'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 198 + [1] * 198) #change number of labels here

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * 132 + [1] * 132)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model_2.h5')

Train on 396 samples, validate on 264 samples
Epoch 1/125
396/396 [==============================] - 1s - loss: 0.7035 - acc: 0.4874 - val_loss: 0.6985 - val_acc: 0.5000
Epoch 2/125
396/396 [==============================] - 0s - loss: 0.7011 - acc: 0.4747 - val_loss: 0.6975 - val_acc: 0.4924
Epoch 3/125
396/396 [==============================] - 0s - loss: 0.6987 - acc: 0.4697 - val_loss: 0.6967 - val_acc: 0.4924
Epoch 4/125
396/396 [==============================] - 0s - loss: 0.6938 - acc: 0.4899 - val_loss: 0.6961 - val_acc: 0.4886
Epoch 5/125
396/396 [==============================] - 0s - loss: 0.6963 - acc: 0.5025 - val_loss: 0.6955 - val_acc: 0.4962
Epoch 6/125
396/396 [==============================] - 0s - loss: 0.7002 - acc: 0.4798 - val_loss: 0.6945 - val_acc: 0.4886
Epoch 7/125
396/396 [==============================] - 0s - loss: 0.6957 - acc: 0.4798 - val_loss: 0.6933 - val_acc: 0.4886
Epoch 8/125
396/396 [==============================] - 0s - loss: 0.6935 - acc: 0.4924